In [43]:
import os
import subprocess
import time
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# Disable SSL
import ssl
import googleapiclient.discovery

ssl._create_default_https_context = ssl._create_unverified_context

# Google API configuration
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = "client_secret.json"
TOKEN_FILE = "token.json"
YOUTUBE_MAX_DURATION = 43200  # 12 hours in seconds

MERGE_TIMEOUT = 1200  # 20 minutes
CONVERT_TIMEOUT = 7200  # 2 hours
MAX_VIDEO_DURATION = 43200  # 12 hours in seconds



In [44]:
def get_authenticated_service():
    """
    Authenticate with YouTube API and return a service object.
    """
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                print("🔄 Token expired. Attempting to refresh...")
                creds.refresh(Request())
                print("✅ Token refreshed successfully!")
            except Exception as e:
                print(f"⚠️ Token refresh failed: {e}\n🌐 Opening browser for re-authentication...")
                creds = None  

        if creds is None:
            print("🌐 Opening browser for authentication...")
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(TOKEN_FILE, "w") as token_file:
            token_file.write(creds.to_json())

    return build("youtube", "v3", credentials=creds)

# 🔥 Call authentication before processing videos
youtube_service = get_authenticated_service()
print("✅ Authentication successful! Now processing videos...")



✅ Authentication successful! Now processing videos...


In [45]:
import subprocess
import os
import re

def get_video_duration(video_path):
    """
    Returns the duration of a video in seconds using GStreamer.
    """
    print(f"📝 Checking file: {video_path}")  # Debug print

    try:
        video_path = os.path.abspath(video_path)  # Ensure absolute path
        result = subprocess.run(
            ["gst-discoverer-1.0", video_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True
        )

        if result.returncode != 0:
            print(f"❌ Error: GStreamer failed with exit code {result.returncode}")
            print(f"⚠️ GStreamer Errors:\n{result.stderr}")
            return None

        # Extract duration using regex
        match = re.search(r"Duration:\s(\d+):(\d+):([\d.]+)", result.stdout)
        if match:
            h, m, s = map(float, match.groups())
            duration = h * 3600 + m * 60 + s
            print(f"✅ Extracted Duration: {duration} seconds")
            return duration

        print("⚠️ Could not extract duration from GStreamer output")
        return None

    except Exception as e:
        print(f"❌ Exception: Could not read duration for {video_path}. Error: {e}")
    
    return None



In [46]:

def convert_videos_gstreamer(folder_path, video_files):
    """
    Converts videos to a uniform format using GStreamer.
    """
    converted_folder = os.path.join(folder_path, "converted_videos")
    os.makedirs(converted_folder, exist_ok=True)
    converted_files = []

    video_files = [f for f in video_files if get_video_duration(f) is not None]
    if not video_files:
        print(f"⚠️ No valid videos to convert in {folder_path}. Skipping...")
        return []

    print(f"🔄 Starting video conversion... (Timeout: {CONVERT_TIMEOUT // 3600} hours)")

    for video in video_files:
        converted_video = os.path.join(converted_folder, os.path.basename(video))
        try:
            subprocess.run(
                [
                    "gst-launch-1.0", "filesrc", f"location={video}", "!", "decodebin",
                    "!", "videoconvert", "!", "videoscale", "!", "video/x-raw,width=1280,height=720",
                    "!", "x264enc", "!", "mp4mux", "!", f"filesink location={converted_video}"
                ],
                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=CONVERT_TIMEOUT
            )
        except subprocess.TimeoutExpired:
            print(f"⏳ Conversion **timed out**! Aborting this subfolder...")
            return []

        converted_files.append(converted_video)

    print("✅ Video conversion complete!")
    return converted_files



In [47]:
import os
import subprocess
import urllib.parse

def merge_videos_gstreamer(folder_path, video_files):
    """
    Merges a list of video files using GStreamer, handling long paths.
    
    :param folder_path: Path to the folder containing videos.
    :param video_files: List of video file paths.
    :return: Path to merged video or None if merge fails.
    """
    if not video_files:
        print("❌ No video files provided for merging.")
        return None

    merged_video_path = os.path.join(folder_path, "merged_video.mp4")

    # Create the playlist file
    playlist_path = os.path.join(folder_path, "playlist.txt")
    with open(playlist_path, "w", encoding="utf-8") as f:
        for video in video_files:
            # Properly encode spaces and special characters
            encoded_video = urllib.parse.quote(os.path.abspath(video))
            f.write(f"file '{encoded_video}'\n")  # Write the file paths in the playlist

    print(f"📜 Playlist created: {playlist_path}")
    print(f"Video files being merged: {video_files}")  # Debugging line to print paths

    # Build the GStreamer command
    gstreamer_command = [
        "gst-launch-1.0", 
        "concat", 
        "filesrc", f"location={playlist_path}", "!", 
        "decodebin", "!", 
        "videoconvert", "!", 
        "x264enc", "!", 
        "mp4mux", "!", 
        f"filesink location={merged_video_path}"
    ]

    print(f"🚀 Running merge command:\n{' '.join(gstreamer_command)}")

    try:
        result = subprocess.run(
            gstreamer_command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True
        )

        if result.returncode == 0:
            print(f"✅ Merge successful: {merged_video_path}")
            return merged_video_path
        else:
            print(f"❌ GStreamer Merge Failed:\n{result.stderr}")
            return None

    except subprocess.CalledProcessError as e:
        print(f"❌ Exception during merge: {e}")
        return None


In [48]:

def merge_videos_and_create_timestamps(folder_path, video_files):
    """
    Merges videos and handles long duration videos.
    """
    print(f"📂 Checking folder: {folder_path}")
    if not os.path.exists(folder_path):
        print(f"❌ Error: The folder path '{folder_path}' does not exist.")
        return None, None, None

    video_files = [f for f in video_files if get_video_duration(f) is not None]
    if not video_files:
        print(f"⚠️ No valid videos found in {folder_path}. Skipping folder...")
        return None, None, None

    total_duration = sum(get_video_duration(f) or 0 for f in video_files)
    if total_duration > MAX_VIDEO_DURATION:
        print(f"⚠️ Video duration too long ({total_duration} sec). Splitting into parts...")
        mid_index = len(video_files) // 2
        first_half = video_files[:mid_index]
        second_half = video_files[mid_index:]

        merged_video_path_1 = merge_videos_gstreamer(folder_path, first_half)
        merged_video_path_2 = merge_videos_gstreamer(folder_path, second_half)

        return [merged_video_path_1, merged_video_path_2], "\n".join(os.path.basename(f) for f in video_files), total_duration

    merged_video_path = merge_videos_gstreamer(folder_path, video_files)
    return merged_video_path, "\n".join(os.path.basename(f) for f in video_files), total_duration


In [49]:

if __name__ == "__main__":
    parent_folder = input("📂 Enter the parent folder containing all video subfolders: ").strip()

    for subfolder in sorted(os.listdir(parent_folder)):
        subfolder_path = os.path.join(parent_folder, subfolder)

        if os.path.isdir(subfolder_path):
            print(f"🚀 Processing folder: {subfolder_path}")

            # Get video files for the current folder
            video_files = sorted([os.path.join(subfolder_path, f) for f in os.listdir(subfolder_path) if f.endswith('.mp4')])

            if not video_files:
                print(f"⚠️ No MP4 files found in {subfolder_path}. Skipping...")
                continue  # Move to the next subfolder

            # Process this folder before moving to the next
            merged_video_path, description, _ = merge_videos_and_create_timestamps(subfolder_path, video_files)

            if merged_video_path:
                upload_to_youtube(merged_video_path, subfolder, description, subfolder_path)

print("✅ All subfolders processed!")



🚀 Processing folder: D:\Adobe\Premiere Pro\New Folder
📂 Checking folder: D:\Adobe\Premiere Pro\New Folder
📝 Checking file: D:\Adobe\Premiere Pro\New Folder\0104.Re-Linking To Assets.mp4
✅ Extracted Duration: 243.72 seconds
📝 Checking file: D:\Adobe\Premiere Pro\New Folder\0201.An Overview Of The Entire Workflow.mp4
✅ Extracted Duration: 191.858333333 seconds
📝 Checking file: D:\Adobe\Premiere Pro\New Folder\0202.Getting A Feel For Non-Linear Editing.mp4
✅ Extracted Duration: 316.88 seconds
📝 Checking file: D:\Adobe\Premiere Pro\New Folder\0104.Re-Linking To Assets.mp4
✅ Extracted Duration: 243.72 seconds
📝 Checking file: D:\Adobe\Premiere Pro\New Folder\0201.An Overview Of The Entire Workflow.mp4
✅ Extracted Duration: 191.858333333 seconds
📝 Checking file: D:\Adobe\Premiere Pro\New Folder\0202.Getting A Feel For Non-Linear Editing.mp4
✅ Extracted Duration: 316.88 seconds
📜 Playlist created: D:\Adobe\Premiere Pro\New Folder\playlist.txt
Video files being merged: ['D:\\Adobe\\Premiere Pr

FileNotFoundError: [WinError 2] The system cannot find the file specified